#### Файл сервера xmlrpc_stats_server_p3.ipynb на Python 3 

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import datetime
import csv
import os
from pathlib import Path

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("127.0.0.1", 8018), requestHandler=RequestHandler)

MAX_COUNT_LINES = 100

# Изменить MAX_COUNT_LINES
def change_limit(limit):
    global MAX_COUNT_LINES
    if isinstance(limit, int) and limit > 0:
        MAX_COUNT_LINES = limit
        return True
    return False
server.register_function(change_limit, 'change_limit')

# Тест
def ping():
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Подсчет строк в файле
def count_lines(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return sum(1 for line in f)
    except FileNotFoundError:
        return 0

# Добавление строки в лог
def add_log(sname):
    log_file = 'logs/logs.csv'
    os.makedirs('logs', exist_ok=True)

    # Получаем текущую временную метку один раз
    current_time = datetime.datetime.now()
    timestamp = current_time.strftime('%Y%m%d_%H%M%S')
    formatted_time = current_time.strftime('%Y-%m-%d %H:%M:%S')

    # Проверяем количество строк
    line_count = count_lines(log_file)
    if line_count >= MAX_COUNT_LINES:
        # Формируем имя архивного файла
        archive_file = f'logs/logs_{timestamp}.csv'
        # Переименовываем текущий файл
        if os.path.exists(log_file):
            os.rename(log_file, archive_file)
        # Создаем новый пустой logs.csv с заголовками
        with open(log_file, 'w', encoding='utf-8', newline='') as f:
            writer = csv.writer(f, delimiter=';')
            writer.writerow(['Event', 'Timestamp'])

    # Добавляем новую запись
    with open(log_file, 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f, delimiter=';')
        writer.writerow([sname, formatted_time])
    return True
server.register_function(add_log, 'add_log')

# Получение содержимого журнала
def get_log():
    log_file = 'logs/logs.csv'
    try:
        with open(log_file, 'r', encoding='utf-8') as f:
            reader = csv.reader(f, delimiter=';')
            log_entries = list(reader)
        return log_entries
    except FileNotFoundError:
        return []
server.register_function(get_log, 'get_log')

print("Listening on port 8018...")
server.serve_forever()

Listening on port 8018...
